In [1]:
# -*- coding: utf-8 -*-
from __future__ import unicode_literals
# text in Western (Windows 1252)

import pickle
import numpy as np
# import StringIO
import math
from keras import optimizers, metrics
from keras.models import Model
from keras.layers import Dense, Dropout, Input
from keras.layers.merge import concatenate
from keras import regularizers
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.constraints import maxnorm
from keras.layers import Flatten
from keras.optimizers import SGD
from keras.models import load_model
# from keras import backend as Input
np.random.seed(7)

Using Theano backend.


In [2]:
# %run ../../../prepare_data.py

import sys
sys.path.insert(0, '../../../')
from prepare_data import *

In [9]:
# %run ../../../prepare_data.py
# X_train, X_other_features_train, y_train, X_test, X_other_features_test, y_test, X_validate, X_other_features_validate, y_validate = generate_full_matrix_inputs('../../internal_representations/inputs/content_shuffle_vector.h5', '../../internal_representations/inputs/shuffle_vector')
# save_inputs('../../internal_representations/inputs/shuffled_letters_train.h5', X_train, y_train, other_features = X_other_features_train)
# save_inputs('../../internal_representations/inputs/shuffled_letters_test.h5', X_test, y_test, other_features = X_other_features_test)
# save_inputs('../../internal_representations/inputs/shuffled_letters_validate.h5', X_validate, y_validate,  other_features = X_other_features_validate)
# X_train, X_other_features_train, y_train = load_inputs('../../internal_representations/inputs/shuffled_letters_train.h5', other_features=True)
# X_test, X_other_features_test, y_test = load_inputs('../../internal_representations/inputs/shuffled_letters_test.h5', other_features=True)
# X_validate, X_other_features_validate, y_validate = load_inputs('../../internal_representations/inputs/shuffled_letters_validate.h5', other_features=True)

data = Data('l')
data.generate_data('letters_word_accetuation_train',
                   'letters_word_accetuation_test',
                   'letters_word_accetuation_validate')

LOADING DATA...
LOAD SUCCESSFUL!


In [4]:
gen2 = data.generator('train', 16)
# test1 = next(gen1)
test2 = next(gen2)

In [7]:
pos = 0
print(len(feature_dictionary))
print(np.array(test2[0][0]).shape)
print(np.array(test2[0][1]).shape)
# for el in test2[0][0]:
# #     print(el)
#     print (data.decode_x(el, dictionary))
#     print(data.decode_x_other_features(feature_dictionary, [el]))

# for el in test2[1]:
#     print(el)

10
(16, 23, 36)
(16, 140)


In [12]:
# print (X_train.shape)
# print (X_test.shape)
# print (X_validate.shape)
pos = 0
# print (decode_input(X_train[pos], dictionary))
# print (decode_X_features(feature_dictionary, [X_other_features_train[pos]]))
# print(decode_position(y_train[pos]))
# print('------------------------------------')
print (data.x_train.shape)
print (data.x_test.shape)
print (data.x_validate.shape)
# pos = 2
print (data.decode_x(data.x_train[pos], dictionary))
print (data.decode_x_other_features(feature_dictionary, [data.x_other_features_train[pos]]))
print(data.x_other_features_train[pos])
print(data.decode_y(data.y_train[pos]))

(430151, 23, 36)
(52058, 23, 36)
(54222, 23, 36)
ašjenlautkajan
Agsnpn-
Agsnpn-
[1 1 0 0 0 1 0 0 1 0 0 1 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 6]
[2, 5]


In [10]:
num_examples = len(data.x_train) # training set size
nn_output_dim = 10
nn_hdim = 516
batch_size = 16
actual_epoch = 40
num_fake_epoch = 20




conv_input_shape=(23, 36)
othr_input = (140, )

conv_input = Input(shape=conv_input_shape, name='conv_input')
x_conv = Conv1D(115, (3), padding='same', activation='relu')(conv_input)
x_conv = Conv1D(46, (3), padding='same', activation='relu')(x_conv)
x_conv = MaxPooling1D(pool_size=2)(x_conv)
x_conv = Flatten()(x_conv)
# x_conv = Dense(516, activation='relu', kernel_constraint=maxnorm(3))(x_conv)

othr_input = Input(shape=othr_input, name='othr_input')
# x_othr = Dense(256, input_dim=167, activation='relu')(othr_input)
# x_othr = Dropout(0.3)(x_othr)
# x_othr = Dense(512, activation='relu')(othr_input)
# x_othr = Dropout(0.3)(x_othr)
# x_othr = Dense(256, activation='relu')(othr_input)

x = concatenate([x_conv, othr_input])
# x = Dense(1024, input_dim=(516 + 256), activation='relu')(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(nn_output_dim, activation='sigmoid')(x)




model = Model(inputs=[conv_input, othr_input], outputs=x)
opt = optimizers.Adam(lr=1E-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=[actual_accuracy,])
# model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

In [11]:
history = model.fit_generator(data.generator('train', batch_size), 
                              data.x_train.shape[0]/(batch_size * num_fake_epoch), 
                              epochs=actual_epoch*num_fake_epoch, 
                              validation_data=data.generator('test', batch_size), 
                              validation_steps=data.x_test.shape[0]/(batch_size * num_fake_epoch),
                              verbose=2,
                              workers=4)

Epoch 1/800
14s - loss: 0.2123 - actual_accuracy: 0.3982 - val_loss: 0.1510 - val_actual_accuracy: 0.6154
Epoch 2/800
16s - loss: 0.1544 - actual_accuracy: 0.5611 - val_loss: 0.1311 - val_actual_accuracy: 0.6392
Epoch 3/800
17s - loss: 0.1404 - actual_accuracy: 0.6073 - val_loss: 0.1224 - val_actual_accuracy: 0.6722
Epoch 4/800
16s - loss: 0.1294 - actual_accuracy: 0.6570 - val_loss: 0.1082 - val_actual_accuracy: 0.7297
Epoch 5/800
17s - loss: 0.1209 - actual_accuracy: 0.6898 - val_loss: 0.1072 - val_actual_accuracy: 0.7542
Epoch 6/800
17s - loss: 0.1143 - actual_accuracy: 0.7075 - val_loss: 0.0948 - val_actual_accuracy: 0.7784
Epoch 7/800
17s - loss: 0.1092 - actual_accuracy: 0.7212 - val_loss: 0.0952 - val_actual_accuracy: 0.7772
Epoch 8/800
18s - loss: 0.1059 - actual_accuracy: 0.7329 - val_loss: 0.0951 - val_actual_accuracy: 0.7753
Epoch 9/800
18s - loss: 0.1007 - actual_accuracy: 0.7492 - val_loss: 0.0904 - val_actual_accuracy: 0.7979
Epoch 10/800
19s - loss: 0.0984 - actual_accur

18s - loss: 0.0222 - actual_accuracy: 0.9491 - val_loss: 0.0804 - val_actual_accuracy: 0.8935
Epoch 79/800
19s - loss: 0.0223 - actual_accuracy: 0.9476 - val_loss: 0.0779 - val_actual_accuracy: 0.8972
Epoch 80/800
19s - loss: 0.0221 - actual_accuracy: 0.9508 - val_loss: 0.0809 - val_actual_accuracy: 0.8903
Epoch 81/800
19s - loss: 0.0217 - actual_accuracy: 0.9498 - val_loss: 0.0863 - val_actual_accuracy: 0.8907
Epoch 82/800
19s - loss: 0.0215 - actual_accuracy: 0.9522 - val_loss: 0.0798 - val_actual_accuracy: 0.8965
Epoch 83/800
20s - loss: 0.0210 - actual_accuracy: 0.9514 - val_loss: 0.0849 - val_actual_accuracy: 0.8861
Epoch 84/800
20s - loss: 0.0212 - actual_accuracy: 0.9518 - val_loss: 0.0749 - val_actual_accuracy: 0.8946
Epoch 85/800
20s - loss: 0.0214 - actual_accuracy: 0.9503 - val_loss: 0.0790 - val_actual_accuracy: 0.8900
Epoch 86/800
19s - loss: 0.0205 - actual_accuracy: 0.9513 - val_loss: 0.0892 - val_actual_accuracy: 0.8869
Epoch 87/800
20s - loss: 0.0195 - actual_accuracy:

20s - loss: 0.0123 - actual_accuracy: 0.9737 - val_loss: 0.0987 - val_actual_accuracy: 0.8935
Epoch 156/800
20s - loss: 0.0114 - actual_accuracy: 0.9746 - val_loss: 0.0955 - val_actual_accuracy: 0.9015
Epoch 157/800
21s - loss: 0.0108 - actual_accuracy: 0.9763 - val_loss: 0.0902 - val_actual_accuracy: 0.9003
Epoch 158/800
19s - loss: 0.0118 - actual_accuracy: 0.9742 - val_loss: 0.0915 - val_actual_accuracy: 0.9003
Epoch 159/800
20s - loss: 0.0117 - actual_accuracy: 0.9740 - val_loss: 0.0949 - val_actual_accuracy: 0.9038
Epoch 160/800
20s - loss: 0.0117 - actual_accuracy: 0.9740 - val_loss: 0.0817 - val_actual_accuracy: 0.9022
Epoch 161/800
20s - loss: 0.0119 - actual_accuracy: 0.9730 - val_loss: 0.0800 - val_actual_accuracy: 0.9103
Epoch 162/800
20s - loss: 0.0114 - actual_accuracy: 0.9750 - val_loss: 0.1001 - val_actual_accuracy: 0.9022
Epoch 163/800
20s - loss: 0.0113 - actual_accuracy: 0.9758 - val_loss: 0.0996 - val_actual_accuracy: 0.8923
Epoch 164/800
20s - loss: 0.0116 - actual_

21s - loss: 0.0081 - actual_accuracy: 0.9823 - val_loss: 0.1043 - val_actual_accuracy: 0.9011
Epoch 232/800
20s - loss: 0.0088 - actual_accuracy: 0.9818 - val_loss: 0.1083 - val_actual_accuracy: 0.9043
Epoch 233/800
20s - loss: 0.0080 - actual_accuracy: 0.9832 - val_loss: 0.1007 - val_actual_accuracy: 0.9041
Epoch 234/800
20s - loss: 0.0084 - actual_accuracy: 0.9818 - val_loss: 0.1084 - val_actual_accuracy: 0.9011
Epoch 235/800
20s - loss: 0.0082 - actual_accuracy: 0.9816 - val_loss: 0.0979 - val_actual_accuracy: 0.9072
Epoch 236/800
21s - loss: 0.0082 - actual_accuracy: 0.9822 - val_loss: 0.1013 - val_actual_accuracy: 0.9076
Epoch 237/800
20s - loss: 0.0076 - actual_accuracy: 0.9834 - val_loss: 0.0962 - val_actual_accuracy: 0.9049
Epoch 238/800
20s - loss: 0.0081 - actual_accuracy: 0.9826 - val_loss: 0.0782 - val_actual_accuracy: 0.9172
Epoch 239/800
20s - loss: 0.0076 - actual_accuracy: 0.9840 - val_loss: 0.1039 - val_actual_accuracy: 0.9095
Epoch 240/800
20s - loss: 0.0081 - actual_

20s - loss: 0.0065 - actual_accuracy: 0.9864 - val_loss: 0.1115 - val_actual_accuracy: 0.8934
Epoch 308/800
20s - loss: 0.0059 - actual_accuracy: 0.9872 - val_loss: 0.1037 - val_actual_accuracy: 0.9095
Epoch 309/800
20s - loss: 0.0058 - actual_accuracy: 0.9877 - val_loss: 0.1210 - val_actual_accuracy: 0.9005
Epoch 310/800
20s - loss: 0.0064 - actual_accuracy: 0.9854 - val_loss: 0.1022 - val_actual_accuracy: 0.9034
Epoch 311/800
19s - loss: 0.0064 - actual_accuracy: 0.9857 - val_loss: 0.1088 - val_actual_accuracy: 0.9041
Epoch 312/800
19s - loss: 0.0066 - actual_accuracy: 0.9857 - val_loss: 0.1047 - val_actual_accuracy: 0.9103
Epoch 313/800
20s - loss: 0.0060 - actual_accuracy: 0.9869 - val_loss: 0.1088 - val_actual_accuracy: 0.8992
Epoch 314/800
20s - loss: 0.0068 - actual_accuracy: 0.9861 - val_loss: 0.1087 - val_actual_accuracy: 0.8999
Epoch 315/800
20s - loss: 0.0068 - actual_accuracy: 0.9847 - val_loss: 0.0884 - val_actual_accuracy: 0.9225
Epoch 316/800
20s - loss: 0.0066 - actual_

...

20s - loss: 0.0047 - actual_accuracy: 0.9899 - val_loss: 0.1134 - val_actual_accuracy: 0.9080
Epoch 460/800
20s - loss: 0.0051 - actual_accuracy: 0.9887 - val_loss: 0.1246 - val_actual_accuracy: 0.8942
Epoch 461/800
20s - loss: 0.0046 - actual_accuracy: 0.9904 - val_loss: 0.1320 - val_actual_accuracy: 0.8884
Epoch 462/800
19s - loss: 0.0044 - actual_accuracy: 0.9911 - val_loss: 0.1202 - val_actual_accuracy: 0.9080
Epoch 463/800
21s - loss: 0.0052 - actual_accuracy: 0.9893 - val_loss: 0.1249 - val_actual_accuracy: 0.9034
Epoch 464/800
21s - loss: 0.0046 - actual_accuracy: 0.9907 - val_loss: 0.1088 - val_actual_accuracy: 0.9068
Epoch 465/800
20s - loss: 0.0045 - actual_accuracy: 0.9903 - val_loss: 0.1348 - val_actual_accuracy: 0.8946
Epoch 466/800
20s - loss: 0.0048 - actual_accuracy: 0.9905 - val_loss: 0.1094 - val_actual_accuracy: 0.9164
Epoch 467/800
20s - loss: 0.0046 - actual_accuracy: 0.9901 - val_loss: 0.1206 - val_actual_accuracy: 0.9030
Epoch 468/800
19s - loss: 0.0044 - actual_

20s - loss: 0.0043 - actual_accuracy: 0.9899 - val_loss: 0.1237 - val_actual_accuracy: 0.9122
Epoch 536/800
20s - loss: 0.0041 - actual_accuracy: 0.9911 - val_loss: 0.1193 - val_actual_accuracy: 0.9118
Epoch 537/800
20s - loss: 0.0037 - actual_accuracy: 0.9921 - val_loss: 0.1275 - val_actual_accuracy: 0.9064
Epoch 538/800
20s - loss: 0.0041 - actual_accuracy: 0.9910 - val_loss: 0.1261 - val_actual_accuracy: 0.9045
Epoch 539/800
20s - loss: 0.0047 - actual_accuracy: 0.9899 - val_loss: 0.1302 - val_actual_accuracy: 0.9008
Epoch 540/800
20s - loss: 0.0046 - actual_accuracy: 0.9903 - val_loss: 0.1324 - val_actual_accuracy: 0.8923
Epoch 541/800
20s - loss: 0.0047 - actual_accuracy: 0.9901 - val_loss: 0.1187 - val_actual_accuracy: 0.9103
Epoch 542/800
20s - loss: 0.0043 - actual_accuracy: 0.9911 - val_loss: 0.1283 - val_actual_accuracy: 0.9076
Epoch 543/800
20s - loss: 0.0046 - actual_accuracy: 0.9891 - val_loss: 0.1068 - val_actual_accuracy: 0.9053
Epoch 544/800
21s - loss: 0.0041 - actual_

20s - loss: 0.0041 - actual_accuracy: 0.9915 - val_loss: 0.1399 - val_actual_accuracy: 0.8923
Epoch 612/800
20s - loss: 0.0039 - actual_accuracy: 0.9924 - val_loss: 0.1295 - val_actual_accuracy: 0.9034
Epoch 613/800
21s - loss: 0.0039 - actual_accuracy: 0.9914 - val_loss: 0.1150 - val_actual_accuracy: 0.9018
Epoch 614/800
20s - loss: 0.0045 - actual_accuracy: 0.9908 - val_loss: 0.1218 - val_actual_accuracy: 0.8896
Epoch 615/800
20s - loss: 0.0037 - actual_accuracy: 0.9916 - val_loss: 0.1279 - val_actual_accuracy: 0.9068
Epoch 616/800
20s - loss: 0.0042 - actual_accuracy: 0.9913 - val_loss: 0.1197 - val_actual_accuracy: 0.9066
Epoch 617/800
20s - loss: 0.0037 - actual_accuracy: 0.9927 - val_loss: 0.1370 - val_actual_accuracy: 0.8934
Epoch 618/800
20s - loss: 0.0035 - actual_accuracy: 0.9921 - val_loss: 0.1342 - val_actual_accuracy: 0.9018
Epoch 619/800
20s - loss: 0.0048 - actual_accuracy: 0.9905 - val_loss: 0.1253 - val_actual_accuracy: 0.8938
Epoch 620/800
21s - loss: 0.0039 - actual_

20s - loss: 0.0036 - actual_accuracy: 0.9922 - val_loss: 0.1310 - val_actual_accuracy: 0.9087
Epoch 688/800
21s - loss: 0.0043 - actual_accuracy: 0.9909 - val_loss: 0.1317 - val_actual_accuracy: 0.8976
Epoch 689/800
21s - loss: 0.0035 - actual_accuracy: 0.9922 - val_loss: 0.1212 - val_actual_accuracy: 0.9011
Epoch 690/800
22s - loss: 0.0035 - actual_accuracy: 0.9926 - val_loss: 0.1234 - val_actual_accuracy: 0.9091
Epoch 691/800
20s - loss: 0.0036 - actual_accuracy: 0.9929 - val_loss: 0.1257 - val_actual_accuracy: 0.9022
Epoch 692/800
20s - loss: 0.0037 - actual_accuracy: 0.9928 - val_loss: 0.1430 - val_actual_accuracy: 0.8980
Epoch 693/800
20s - loss: 0.0045 - actual_accuracy: 0.9906 - val_loss: 0.1272 - val_actual_accuracy: 0.9049
Epoch 694/800
21s - loss: 0.0039 - actual_accuracy: 0.9910 - val_loss: 0.1336 - val_actual_accuracy: 0.8978
Epoch 695/800
21s - loss: 0.0037 - actual_accuracy: 0.9920 - val_loss: 0.1265 - val_actual_accuracy: 0.9076
Epoch 696/800
20s - loss: 0.0036 - actual_

20s - loss: 0.0039 - actual_accuracy: 0.9918 - val_loss: 0.0914 - val_actual_accuracy: 0.9149
Epoch 764/800
20s - loss: 0.0039 - actual_accuracy: 0.9922 - val_loss: 0.1217 - val_actual_accuracy: 0.9149
Epoch 765/800
20s - loss: 0.0035 - actual_accuracy: 0.9929 - val_loss: 0.1197 - val_actual_accuracy: 0.8980
Epoch 766/800
21s - loss: 0.0032 - actual_accuracy: 0.9932 - val_loss: 0.1209 - val_actual_accuracy: 0.9137
Epoch 767/800
22s - loss: 0.0037 - actual_accuracy: 0.9923 - val_loss: 0.1092 - val_actual_accuracy: 0.9149
Epoch 768/800
21s - loss: 0.0035 - actual_accuracy: 0.9928 - val_loss: 0.1223 - val_actual_accuracy: 0.9045
Epoch 769/800
20s - loss: 0.0041 - actual_accuracy: 0.9921 - val_loss: 0.1392 - val_actual_accuracy: 0.8972
Epoch 770/800
20s - loss: 0.0036 - actual_accuracy: 0.9925 - val_loss: 0.1252 - val_actual_accuracy: 0.9095
Epoch 771/800
21s - loss: 0.0037 - actual_accuracy: 0.9927 - val_loss: 0.1332 - val_actual_accuracy: 0.8912
Epoch 772/800
21s - loss: 0.0040 - actual_

In [8]:
name = '40_epoch'
model.save(name + '.h5')
output = open(name + '_history.pkl', 'wb')
pickle.dump(history.history, output)
output.close()

In [7]:
content = data._read_content('../../../data/SlovarIJS_BESEDE_utf8.lex')
dictionary, max_word, max_num_vowels, vowels, accented_vowels = data._create_dict(content)
feature_dictionary = data._create_feature_dictionary()